# Chapter 8: Orchestration — Putting It All Together

Now we combine everything: tools for data, code for analysis, and memory for context. The agent **orchestrates** multiple tools and computations to answer complex portfolio questions.

This is the bridge between the simpler patterns (Chapters 5-7) and the full portfolio optimization system (Notebook 05). Instead of hardcoded dictionaries, we use **live market data** via `yfinance` — the same data source you would use in a real application.

**What the agent will do:**
- **Tool calling** to fetch real prices, returns, and sector data
- **ReAct reasoning** to plan which data it needs and in what order
- **CodeAct execution** to calculate portfolio metrics (weights, volatility, risk contribution)
- **Memory** to maintain context across follow-up questions

This is **orchestration**: the agent deciding which pattern to use at each step, combining them fluidly to solve a problem no single pattern could handle alone.

In [1]:
!pip install smolagents yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel

In [3]:
import getpass
API_KEY = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [4]:
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model initialized!")

Model initialized!


## Building the Tools

Three tools for our portfolio assistant:

1. **`get_stock_price`** — Current market price for any ticker
2. **`get_historical_returns`** — Last 12 months of monthly returns (for risk/return analysis)
3. **`get_company_sector`** — Sector classification (for diversification analysis)

All three use `yfinance` to pull live data from Yahoo Finance. This means results will change each time you run the notebook — just like real portfolio analysis.

In [5]:
@tool
def get_stock_price(ticker: str) -> float:
    """Get the current price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The current stock price as a float
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('regularMarketPrice', 0.0)

@tool
def get_historical_returns(ticker: str) -> list:
    """Get historical monthly returns for a stock (last 12 months).

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        List of monthly return percentages as decimals
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y", interval="1mo")
    returns = hist['Close'].pct_change().dropna().tolist()
    return returns

@tool
def get_company_sector(ticker: str) -> str:
    """Get the sector classification for a company.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The company's sector as a string
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('sector', 'Unknown')

print("Three tools created: get_stock_price, get_historical_returns, get_company_sector")

Three tools created: get_stock_price, get_historical_returns, get_company_sector


## Creating the Orchestrated Agent

The agent has three tools plus code execution capabilities. With `max_steps=15`, it can handle complex multi-step analyses that require gathering data from multiple sources before computing results.

The `additional_authorized_imports` gives it `numpy` and `pandas` for computations — the agent can write and execute code that uses these libraries to calculate portfolio metrics like weighted returns, volatility, and risk contributions.

In [6]:
portfolio_assistant = CodeAgent(
    tools=[get_stock_price, get_historical_returns, get_company_sector],
    model=model,
    verbosity_level=LogLevel.INFO,
    max_steps=15,
    additional_authorized_imports=["numpy", "pandas"]
)

print("Portfolio assistant ready!")

Portfolio assistant ready!


## Orchestration in Action

Watch how the agent coordinates multiple tool calls and computations to answer a complex portfolio analysis question. A single prompt triggers a chain of actions:

1. **Tool calls** to fetch prices, returns, and sectors for each stock
2. **Code execution** to calculate position values, weights, and volatility
3. **Reasoning** to synthesize findings into a coherent analysis

The agent decides the order and approach — you just ask the question.

In [7]:
result = portfolio_assistant.run("""
Analyze this portfolio:
- 100 shares of AAPL
- 50 shares of NVDA
- 200 shares of MSFT

Calculate:
1. Current value of each position and total portfolio value
2. Portfolio weights (% allocation to each stock)
3. Sector concentration
4. Which stock has the highest historical volatility?
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Analyze this portfolio:                                                                                         │
│ - 100 shares of AAPL                                                                                            │
│ - 50 shares of NVDA                                                                                             │
│ - 200 shares of MSFT                                                                                            │
│                                                                                                                 │
│ Calculate:                                                                                                      │
│ 1. Current value of each position and total portfolio value                                                     │
│ 2. Portfolio weights (% allocation to each stock)                                                               │
│ 3. Sector concentration                                                                                         │
│ 4. Which stock has the highest historical volatility?                                                           │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  aapl_price = get_stock_price('AAPL')                                                                             
  nvda_price = get_stock_price('NVDA')                                                                             
  msft_price = get_stock_price('MSFT')                                                                             
                                                                                                                   
  print(f"AAPL price: {aapl_price}, NVDA price: {nvda_price}, MSFT price: {msft_price}")                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
AAPL price: 262.74, NVDA price: 187.92, MSFT price: 397.5075

Out: None

[Step 1: Duration 8.58 seconds| Input tokens: 2,197 | Output tokens: 233]

...

[Step 6: Duration 8.46 seconds| Input tokens: 19,901 | Output tokens: 1,573]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "current_values": {                                                                                          
          "AAPL": 26274.0,                                                                                         
          "NVDA": 9396.0,                                                                                          
          "MSFT": 79501.5,                                                                                         
          "total_portfolio_value": 115171.5                                                                        
      },                                                                                                           
      "portfolio_weights": {                                                                                       
          "AAPL": 22.81,                                                                                           
          "NVDA": 8.16,                                                                                            
          "MSFT": 69.03                                                                                            
      },                                                                                                           
      "sector_concentration": "Technology",                                                                        
      "highest_volatility_stock": "NVDA"                                                                           
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'current_values': {'AAPL': 26274.0, 'NVDA': 9396.0, 'MSFT': 79501.5, 'total_portfolio_value': 
115171.5}, 'portfolio_weights': {'AAPL': 22.81, 'NVDA': 8.16, 'MSFT': 69.03}, 'sector_concentration': 'Technology',
'highest_volatility_stock': 'NVDA'}

[Step 7: Duration 6.21 seconds| Input tokens: 25,318 | Output tokens: 1,905]

## Inspecting the Orchestration

Let's look at exactly what the agent did -- which tools it called, in what order, and how it combined the results. This is the orchestration trace: the sequence of decisions the agent made to solve the problem.

In [8]:
print("Agent Memory:")
print("=" * 50)
for i, step in enumerate(portfolio_assistant.memory.steps):
    step_type = type(step).__name__
    print(f"\nStep {i+1}: {step_type}")

    if step_type == "TaskStep":
        print(f"   Task: {step.task}")

    elif step_type == "ActionStep":
        if hasattr(step, 'tool_calls') and step.tool_calls:
            for tc in step.tool_calls:
                print(f"   Tool: {tc.name}({tc.arguments})")

Agent Memory:

Step 1: TaskStep
   Task: 
Analyze this portfolio:
- 100 shares of AAPL
- 50 shares of NVDA
- 200 shares of MSFT

Calculate:
1. Current value of each position and total portfolio value
2. Portfolio weights (% allocation to each stock)
3. Sector concentration
4. Which stock has the highest historical volatility?


Step 2: ActionStep
   Tool: python_interpreter(aapl_price = get_stock_price('AAPL')
nvda_price = get_stock_price('NVDA')
msft_price = get_stock_price('MSFT')

print(f"AAPL price: {aapl_price}, NVDA price: {nvda_price}, MSFT price: {msft_price}"))

Step 3: ActionStep
   Tool: python_interpreter(# Given number of shares
shares_aapl = 100
shares_nvda = 50
shares_msft = 200

# Calculate current values of each position
value_aapl = shares_aapl * aapl_price
value_nvda = shares_nvda * nvda_price
value_msft = shares_msft * msft_price

# Total portfolio value
total_portfolio_value = value_aapl + value_nvda + value_msft

# Print the current values and total portfolio valu

## Follow-Up with Memory

Using `reset=False`, the agent remembers the previous analysis. We can ask follow-up questions that build on what it already computed — no need to repeat the portfolio details or re-fetch the data.

This is **conversational context** in action. The agent maintains state across interactions, just like a human analyst who remembers what you discussed five minutes ago.

In [9]:
result = portfolio_assistant.run("""
What if I sold 25 shares of NVDA and bought more MSFT with the proceeds?
Show the before and after portfolio comparison.
""", reset=False)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What if I sold 25 shares of NVDA and bought more MSFT with the proceeds?                                        │
│ Show the before and after portfolio comparison.                                                                 │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Number of shares to sell and current price of NVDA                                                             
  shares_to_sell_nvda = 25                                                                                         
  sale_proceeds_nvda = shares_to_sell_nvda * nvda_price                                                            
                                                                                                                   
  # Print the sale proceeds                                                                                        
  print(f"Sale proceeds from selling {shares_to_sell_nvda} shares of NVDA: {sale_proceeds_nvda}")                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Sale proceeds from selling 25 shares of NVDA: 4698.0

Out: None

[Step 8: Duration 4.77 seconds| Input tokens: 31,378 | Output tokens: 2,102]

...

[Step 9: Duration 6.51 seconds| Input tokens: 37,777 | Output tokens: 2,476]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "before_transaction": {                                                                                      
          "AAPL": {"shares": 100, "value": 26274.0},                                                               
          "NVDA": {"shares": 50, "value": 9396.0},                                                                 
          "MSFT": {"shares": 200, "value": 79501.5},                                                               
          "total_portfolio_value": 115171.5                                                                        
      },                                                                                                           
      "after_transaction": {                                                                                       
          "AAPL": {"shares": 100, "value": 26274.0},                                                               
          "NVDA": {"shares": 25, "value": 4698.0},                                                                 
          "MSFT": {"shares": 211, "value": 83874.0825},                                                            
          "total_portfolio_value": 114846.0825                                                                     
      }                                                                                                            
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'before_transaction': {'AAPL': {'shares': 100, 'value': 26274.0}, 'NVDA': {'shares': 50, 'value': 
9396.0}, 'MSFT': {'shares': 200, 'value': 79501.5}, 'total_portfolio_value': 115171.5}, 'after_transaction': 
{'AAPL': {'shares': 100, 'value': 26274.0}, 'NVDA': {'shares': 25, 'value': 4698.0}, 'MSFT': {'shares': 211, 
'value': 83874.0825}, 'total_portfolio_value': 114846.0825}}

[Step 10: Duration 10.34 seconds| Input tokens: 44,984 | Output tokens: 2,989]

In [10]:
result = portfolio_assistant.run("""
Which position contributes most to portfolio volatility?
Should I consider reducing it for better risk management?
""", reset=False)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which position contributes most to portfolio volatility?                                                        │
│ Should I consider reducing it for better risk management?                                                       │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Previous values                                                                                                
  weights_before = {                                                                                               
      "AAPL": 0.22812935491853453,                                                                                 
      "NVDA": 0.08158268321589977,                                                                                 
      "MSFT": 0.6902879618655657                                                                                   
  }                                                                                                                
  volatilities = {                                                                                                 
      "AAPL": 0.055220855868450276,                                                                                
      "NVDA": 0.0956205711831595,                                                                                  
      "MSFT": 0.07678118931653634                                                                                  
  }                                                                                                                
                                                                                                                   
  # Calculate total portfolio volatility                                                                           
  portfolio_volatility = np.sqrt(                                                                                  
      sum(weights_before[stock]**2 * volatilities[stock]**2 for stock in weights_before)                           
  )                                                                                                                
                                                                                                                   
  # Calculate contribution of each stock to portfolio volatility                                                   
  contributions = {stock: weights_before[stock]**2 * volatilities[stock] for stock in weights_before}              
                                                                                                                   
  # Identify the stock with the highest contribution to volatility                                                 
  highest_contribution_stock = max(contributions, key=contributions.get)                                           
  highest_contribution_value = contributions[highest_contribution_stock]                                           
                                                                                                                   
  # Print the results                                                                                              
  print(f"Total portfolio volatility: {portfolio_volatility}")                                                     
  print(f"Contributions to portfolio volatility: {contributions}")                                                 
  print(f"Stock with the highest contribution to volatility: {highest_contribution_stock}                          
  ({highest_contribution_value})")                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Total portfolio volatility: 0.05503337303308584
Contributions to portfolio volatility: {'AAPL': 0.0028738591441856507, 'NVDA': 0.0006364251059147834, 'MSFT': 
0.0365860424756875}
Stock with the highest contribution to volatility: MSFT (0.0365860424756875)

Out: None

[Step 11: Duration 9.75 seconds| Input tokens: 53,114 | Output tokens: 3,511]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "total_portfolio_volatility": 0.05503337303308584,                                                           
      "contributions": {                                                                                           
          "AAPL": 0.00287,                                                                                         
          "NVDA": 0.00064,                                                                                         
          "MSFT": 0.03659                                                                                          
      },                                                                                                           
      "highest_contribution_stock": "MSFT",                                                                        
      "recommendation": "Consider reducing position in MSFT for better risk management."                           
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'total_portfolio_volatility': 0.05503337303308584, 'contributions': {'AAPL': 0.00287, 'NVDA': 
0.00064, 'MSFT': 0.03659}, 'highest_contribution_stock': 'MSFT', 'recommendation': 'Consider reducing position in 
MSFT for better risk management.'}

[Step 12: Duration 4.53 seconds| Input tokens: 62,198 | Output tokens: 3,744]

## Exercise

Create a multi-step portfolio analysis question that requires the agent to use all three tools and perform custom calculations.

**Ideas to try:**
- "Calculate the risk-adjusted return (Sharpe ratio) for each stock and the overall portfolio"
- "How diversified is this portfolio? What would adding GOOGL and AMZN do?"
- "What's the optimal position sizing if I want to minimize volatility while keeping expected return above X%?"
- "Compare the historical performance of equal-weight vs. current allocation"

**Challenge:** Can you get the agent to combine all three tools *and* write numpy/pandas code in a single analysis?

In [ ]:
result = portfolio_assistant.run("""
    YOUR PROMPT HERE
""")